In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
from pathlib import Path

sys.path.append(str(Path().cwd().parent))

In [4]:
from typing import Tuple

import pandas as pd
import numpy as np
import isodate

from plotting import plot_ts
from load_dataset import Dataset
from model import TimeSeriesPredictor, get_timedelta_from_granularity
import datetime as dt

/home/nikita/Документы/time_series_course/ts_march_2023/model.py:233: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  def predict_batch(self, ts: pd.Series, ts_batch: pd.Series = pd.Series()):


### Какие ряды будем тестировать?

* длинный ряд с сезонностью  
* короткий ряд с сезонностью  
* короткий ряд с сезонностью и трендом  
* случайное блуждание  
* средне зашумленный ряд
* "шумный" ряд  

In [5]:
ds = Dataset('../data/dataset/')

In [6]:
long = ds['daily-min-temperatures.csv']

/home/nikita/Документы/time_series_course/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [7]:
plot_ts(long)

Loading BokehJS ...

In [8]:
short_season = ds['hour_3019.csv'][300:]

/home/nikita/Документы/time_series_course/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [9]:
plot_ts(short_season)

Loading BokehJS ...

In [10]:
short_season_trend = ds['international-airline-passengers.csv']

/home/nikita/Документы/time_series_course/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [11]:
plot_ts(short_season_trend)

Loading BokehJS ...

In [12]:
random_walk = ds['dow_jones_0.csv']

/home/nikita/Документы/time_series_course/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [13]:
plot_ts(random_walk)

Loading BokehJS ...

In [14]:
medium_noize = ds['hour_3426.csv'][300:]

/home/nikita/Документы/time_series_course/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [15]:
plot_ts(medium_noize)

Loading BokehJS ...

In [16]:
full_noize = ds['day_1574.csv']

/home/nikita/Документы/time_series_course/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [17]:
plot_ts(full_noize)

Loading BokehJS ...

### Какие модели будем тестировать?

* скользящее среднее
* экспоненциальное сглаживание
* autoArima
* линейная регрессия
* линейная регрессия с L1 регуляризацией (Ridge)
* RandomForeset
* градиентный бустинг
* полносвязная нейросеть с одним лагом в качестве горизонта прогнозирования
* полносвязная нейросеть с произвольным количеством лагов в качестве горизонта прогнозирования

In [18]:
from estimators import RollingEstimator, ExponentialSmoothingEstimator
from pmdarima import auto_arima
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

### По каким метрикам будем сравнивать?

* mse
* mae
* R2
* mape - если не будет ломаться на нулях
* mase

In [19]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import r2_score

from metrics import mase

### По какой методике будем тестировать?

* 70% трейн, 30% тест
* Out-of-sample, чтобы посмотреть как модель предсказывает "вдолгую"

`ВНИМАНИЕ`
Чтобы сделать корректный прогноз на тестовую выборку в режиме out-of-sample с учетом возможных пропусков, вам необходимо сделать прогноз от первого до последнего timestamp-a включительно, после чего взять нужные timestamp из теста (реализуй это в функции задания 1б).  
Нельзя просто написать `predictor.forecast(len(test))`.


* In-Sample, чтобы посмотреть как модель предсказывает на одну точку вперед
* Для поиска гиперпараметров можно делать кроссвалидацию на тесте по метрике mse

### Задание 1(а). Напишите функцию, разбивающую на train и test

In [20]:
def train_test_split(ts: pd.Series, ratio: float = 0.7) -> Tuple[pd.Series]:
    # ваш код здесь
    split_idx = int(len(ts) * ratio)
    ts_train, ts_test = ts[:split_idx], ts[split_idx:]
    return ts_train, ts_test

### Задание 1(б). Напишите функцию для получения численного горизонта прогнозирования через первый и последний timestamp-ы выборки.

В данной функции горизонт прогнозирования задается первым и последним timestamp-ом, а также гранулярностью ряда. Нам нужно получить горизонт прогнозирования в виде целового числа лагов, на которые нужно сделать прогноз.

In [21]:
def calculate_h(start: pd.Timestamp, end: pd.Timestamp, granularity: str) -> int:
    # ваш код здесь
    h = (end - start) / isodate.parse_duration(granularity)
    return int(h)

### Зададим соответствие гранулярностей для наших рядов.

In [22]:
granularity_mapping = {
    'long': 'P1D',
    'short_season': 'PT1H',
    'short_season_trend': 'P1M',
    'random_walk': 'P1D',
    'medium_noize': 'PT1H',
    'full_noize': 'P1D'
}

In [23]:
import math

from pandas._libs.tslibs.timestamps import Timestamp


def get_month_sin(timestamp: Timestamp) -> float:
    theta = timestamp.month * (2*math.pi / 12)
    return math.sin(theta)


def get_month_cos(timestamp: Timestamp) -> float:
    theta = timestamp.month * (2*math.pi / 12)
    return math.cos(theta)


def get_day_sin(timestamp: Timestamp) -> float:
    theta = timestamp.day * (2*math.pi / timestamp.days_in_month)
    return math.sin(theta)


def get_day_cos(timestamp: Timestamp) -> float:
    theta = timestamp.day * (2*math.pi / timestamp.days_in_month)
    return math.cos(theta)


def get_dayofweek_sin(timestamp: Timestamp) -> float:
    theta = timestamp.dayofweek * (2*math.pi / 7)
    return math.sin(theta)


def get_dayofweek_cos(timestamp: Timestamp) -> float:
    theta = timestamp.dayofweek * (2*math.pi / 7)
    return math.cos(theta)


def get_hour_sin(timestamp: Timestamp) -> float:
    theta = timestamp.hour * (2*math.pi / 24)
    return math.sin(theta)


def get_hour_cos(timestamp: Timestamp) -> float:
    theta = timestamp.hour * (2*math.pi / 24)
    return math.cos(theta)


def get_minute_sin(timestamp: Timestamp) -> float:
    theta = timestamp.minute * (2*math.pi / 60)
    return math.sin(theta)


def get_minute_cos(timestamp: Timestamp) -> float:
    theta = timestamp.minute * (2*math.pi / 60)
    return math.cos(theta)


datetime_mappers = {
    'month_sin': get_month_sin,
    'month_cos': get_month_cos,
    'day_sin': get_day_sin,
    'day_cos': get_day_cos,
    'dayofweek_sin': get_dayofweek_sin,
    'dayofweek_cos': get_dayofweek_cos,
    'hour_sin': get_hour_sin,
    'hour_cos': get_hour_cos,
    'minute_sin': get_minute_sin,
    'minute_cos': get_minute_cos,
}

### Задание 2. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через TimeSeriesPredictor.

* принмает на вход исходный ряд, гранулярность, количество лагов, модель, а также **kwargs, в которые мы будем передавать параметры модели

* разбивает ряд на train/test

* создает инстанс TimeSeriesPredictor с нужными параметрами

* обучает предиктор на трейне

* делает out_of_sample и in_sample прогноз

* возвращает train, test, in_sample, out_of_sample

In [24]:
def make_pipeline(
    ts: pd.Series,
    granularity: str,
    model: callable,
    num_lags=24,
    use_mappers=True,
    **kwargs
) -> Tuple[pd.Series]:
    
    # your code here
    ts_train, ts_test = train_test_split(ts)
        
    predictor = TimeSeriesPredictor(granularity, num_lags, model, **kwargs)
    if use_mappers:
        predictor.set_params(mappers = datetime_mappers)
    
    predictor.fit(ts_train, **kwargs)
    
    in_sample = predictor.predict_batch(ts_train, ts_test)
    
    
    horizon = calculate_h(ts_train.index[-1], ts_test.index[-1], granularity = granularity)
    out_of_sample = predictor.predict_next(ts_train, n_steps = horizon)
    out_of_sample = out_of_sample[ts_test.index] #только то, что есть в тесте
    
    return ts_train, ts_test, in_sample, out_of_sample

### Задание 3. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через auto_arima

* функция должна принимать исходный временной ряд, период сезонности, параметры дифференцирования d, D и boolean параметр seasonal, данные параметры будут являться для нас гиперпараметрами, все остальное за нас должна найти auto_arima

* разбивает на train, test

* обучает arima на train при помощи вызова функции auto_arima из библиотеки pmdarima с переданными параметрами и со следующими зафиксированными параметрами: `max_p=3, max_q=3, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True`

* в качестве out_of_sample прогноза просто вызовите метод predict

* в качестве in_sample прогноза обучите модель заново на всём ряде методом `fit`, вызовите метод predict_in_sample и в качестве прогноза возьмите `in_sample_predictions(-len(test):)`

* возвращает train, test, in_sample, out_of_sample (не забудьте сделать их pd.Series с нужным индексом!!)

In [25]:
from pmdarima import auto_arima

In [26]:
def make_pipeline_arima(ts: pd.Series, granularity: str, period: int = 1, d: int = 1,
                        D: int = 1, seasonal: bool = True) -> Tuple[pd.Series]:
    # your code here
    
    ts_train, ts_test = train_test_split(ts)
    
    model = auto_arima(
     ts_train, start_p=0, start_q=0,
     max_p=3, max_q=3, m=period,
     start_P=0, start_Q=0, seasonal=seasonal,
     d=d, D=D, trace=True,
     error_action='ignore',
     suppress_warnings=True,
     stepwise=True)
    
    delta = get_timedelta_from_granularity(granularity)
    horizon = calculate_h(ts_train.index[-1], ts_test.index[-1], granularity = granularity)
    ind = np.arange(ts_train.index[-1] + delta, ts_test.index[-1] + delta, delta).astype(dt.datetime)
    values = model.predict(horizon).values
    out_of_sample = pd.Series(data = values, index = ind)
    
    model.fit(ts)
    in_sample = model.predict_in_sample()[-len(ts_test):]

    return ts_train, ts_test, in_sample, out_of_sample[ts_test.index]

In [28]:
ts_train, ts_test, in_sample, out_of_sample = make_pipeline_arima(random_walk, granularity = 'P0Y0M1D', seasonal = False, d = 1, D = 0)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2369.004, Time=0.27 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2368.625, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2368.521, Time=0.26 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2367.012, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2370.526, Time=0.12 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.733 seconds


/home/nikita/Документы/time_series_course/ts_env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


### Задание 4. Напишите функцию, имплементирующую весь пайплайн обучения и прогноза через полносвязную сеть.

* функция должна принимать исходный временной ряд, количество входных лагов для формирования признаков, количество выходных лагов для формирования таргетов, количество скрытых слоев и количество нейронов на каждом слое
* подготавилвает выборку согласно переданным параметрам num_lags_in, num_lags_out, используя ранее написанную нами функцию transform_ts_into_matrix (приведена ниже)
* разбивает данные на трейн и тест
* создает модель Sequential с нужной архитектурой
    - num_lags_in задает значение параметра input_dims на первом слое
    - num_lags_out задает количество нейронов на последнем слое
    - количество нейронов на всех слоях от первого до предпоследнего задается в кортеже units
    - epochs задает количество епох для обучения
* обучает модель на трейне
* делает in_sample прогноз на тесте вызовом метода predict (обратите внимание, что вызов предикта должен осуществляться с шагом num_lags_out)
* делает out_of_sample прогноз рекурсивно c шагом num_lags_out, добавляя спрогнозированные точки в новые объекты (см. аналогично TimeSeriesPredictor)

In [29]:
def transform_ts_into_matrix(ts: pd.Series, num_lags_in: int, num_lags_out: int) -> Tuple[np.array, np.array]:
    """
    Данная функция должна пройтись скользящим окном по временному ряду и для каждых
    num_lags_in точек в качестве признаков собрать num_lags_out следующих точек в качестве таргета.
    
    Вернуть два np.array массива из X_train и y_train соответственно
    """
    sequence = ts.values
    X, y = list(), list()
    i = 0
    outer_idx = num_lags_out
    while outer_idx < len(sequence):
        inner_idx = i + num_lags_in
        outer_idx = inner_idx + num_lags_out
        X_, y_ = sequence[i:inner_idx], sequence[inner_idx:outer_idx]
        X.append(X_)
        y.append(y_)
        i += 1
    return np.array(X), np.array(y)

In [44]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense


def make_pipeline_fullyconnected(
    ts: pd.Series,
    granularity: str,
    num_lags_in: int,
    num_lags_out: int,
    units: tuple[int],
    epochs: int,
    ratio: float = 0.7
) -> Tuple[pd.Series]:
    # your code here
    train, test = train_test_split(ts, ratio = ratio)
    
    
    X_train, y_train = transform_ts_into_matrix(train, num_lags_in = num_lags_in, num_lags_out = num_lags_out)
    X_test, y_test = transform_ts_into_matrix(test, num_lags_in = num_lags_in, num_lags_out = num_lags_out)
    
    nn = Sequential()
    nn.add(Dense(units[0], activation="relu", input_dim=num_lags_in))
    for n_units in units[1:]:
        nn.add(Dense(n_units, activation="relu"))
        
    nn.add(Dense(num_lags_out))
    nn.compile(loss='mse', optimizer='adam')
    print(nn.summary())
    nn.fit(X_train, y_train, epochs=epochs, verbose=1)

        #in_sample. Делается предсказание на n_lags_out точек вперед по точно известным n_lags_in предыдущим
    df = ts.copy()
    split_idx = int(len(df)*ratio)
    cur_idx = split_idx
    in_sample = pd.Series(dtype = float)
    delta = get_timedelta_from_granularity(granularity)
    
    while len(in_sample) < len(test):
        new_X = [df[cur_idx-i] for i in range(num_lags_in, 0, -1)]
        new_preds = nn.predict([new_X],verbose = 0).flatten()
        in_sample = pd.concat([in_sample, pd.Series(new_preds)])
        cur_idx += num_lags_out
#         print(len(test), cur_idx, len(ts))
#         print(len(in_sample), len(test))
    in_sample = in_sample[:len(test)]
    in_sample.index = test.index
        
    
    #out_of_sample. Тоже предсказание на n_lags_out точек вперед, но по предсказанным предыдущим
    out_of_sample = pd.Series(dtype = float)
    h = calculate_h(train.index[-1], test.index[-1], granularity = granularity)
    df = train.copy()
    
    while len(out_of_sample) < h:
        new_X = [df[-i] for i in range(num_lags_in, 0, -1)]
        new_preds = nn.predict([new_X],verbose = 0).flatten()
        
        ind = [df.index[-1] + delta*i for i in range(1, num_lags_out+1)]
        
        new_preds = pd.Series(new_preds, index = ind)
        out_of_sample = pd.concat([out_of_sample, new_preds])
        df = pd.concat([df, new_preds])
    
    out_of_sample = out_of_sample[test.index]
        
    return train, test, in_sample, out_of_sample    

In [31]:
# train, test, in_sample, out_of_sample = make_pipeline_fullyconnected(random_walk, 'P0Y0M1D' ,24, 12, 2, (64,32, 12), 2000)

In [32]:
# plot_ts(train, test, in_sample, out_of_sample)

### Задание 5. Напишите функцию, имплементирующую поиск гиперпараметров по сетке. 

* функция должна принимать на вход ряд, гранулярность, модель, дефолтное количество лагов, сетку параметров (словарь)
* после написанного мной кода, функция должна с текущими параметрами запустить пайплайн (функция make_pipeline), получив таким образом прогнозы in_sample и out_of_sample
* посчитать mse для in_sample и out_of_sample прогноза, запомнить их в соответствующие словари
* вернуть лучшие параметры для in_sample и out_of_sample прогнозов

Замечания
* не забудьте, что в сетку параметров можно передавать также num_lags
* если в ряде ts_test есть пропуски, индекс прогноза out_of_sample будет не совпадать c индексом реальных данных, в таком случае, замените индекс out_of_sample прогноза индексом ts_test

In [33]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [144]:
from itertools import product

def hyperparameters_search(ts, granularity, model, num_lags, param_grid, verbose=False, use_mappers=True):
    
    min_in_sample, min_out_of_sample = np.inf, np.inf
    best_kwargs_in_sample, best_kwargs_out_of_sample = {}, {}
    in_sample_best, out_of_sample_best = [], []
    
    for param_tuple in product(*param_grid.values()):
        
        kwargs = {key: value for key, value in zip(param_grid.keys(), param_tuple)}
        num_lags = kwargs['num_lags']
        use_mappers = kwargs['use_mappers']
        del kwargs['num_lags']
        del kwargs['use_mappers']
        
        ts_train, ts_test, in_sample, out_of_sample = make_pipeline(ts,
                                                                    granularity, model,
                                                                    num_lags, use_mappers, **kwargs)
        loss_in_sample = mse(ts_test, in_sample)       
        loss_out_of_sample = mse(ts_test, out_of_sample)
        
        
        kwargs['num_lags'] = num_lags
        kwargs['use_mappers'] = use_mappers
        
        if loss_in_sample < min_in_sample:
            best_kwargs_in_sample = kwargs
            in_sample_best = in_sample
            
        if loss_out_of_sample < min_out_of_sample:
            best_kwargs_out_of_sample = kwargs
            out_of_sample_best = out_of_sample
            
        if verbose:
            print(f"{kwargs}, mse in sample: {loss_in_sample:.2f}, mse_ot_of_sample: {loss_out_of_sample:.2f}")
        
    
    return in_sample_best, best_kwargs_in_sample, min_in_sample, out_of_sample_best, best_kwargs_out_of_sample, min_out_of_sample 

In [145]:
params = {'num_lags': [7,15,30, 50],
         'use_mappers': [True, False],
         'alpha': [0.2, 0.5, 1, 1.5]}

In [139]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [146]:
a,b,c,d,e,f = hyperparameters_search(random_walk,'P0Y0M1D',Ridge, 12, params, verbose = True)

{'alpha': 0.2, 'num_lags': 7, 'use_mappers': True}, mse in sample: 60827.38, mse_ot_of_sample: 5581922.01
{'alpha': 0.5, 'num_lags': 7, 'use_mappers': True}, mse in sample: 60462.88, mse_ot_of_sample: 5548921.10
{'alpha': 1, 'num_lags': 7, 'use_mappers': True}, mse in sample: 59879.33, mse_ot_of_sample: 5494960.38
{'alpha': 1.5, 'num_lags': 7, 'use_mappers': True}, mse in sample: 59323.97, mse_ot_of_sample: 5442262.99
{'alpha': 0.2, 'num_lags': 7, 'use_mappers': False}, mse in sample: 40280.23, mse_ot_of_sample: 1334941.72
{'alpha': 0.5, 'num_lags': 7, 'use_mappers': False}, mse in sample: 40280.23, mse_ot_of_sample: 1334941.74
{'alpha': 1, 'num_lags': 7, 'use_mappers': False}, mse in sample: 40280.23, mse_ot_of_sample: 1334941.76
{'alpha': 1.5, 'num_lags': 7, 'use_mappers': False}, mse in sample: 40280.22, mse_ot_of_sample: 1334941.79
{'alpha': 0.2, 'num_lags': 15, 'use_mappers': True}, mse in sample: 52883.22, mse_ot_of_sample: 1835112.03
{'alpha': 0.5, 'num_lags': 15, 'use_mappers':

In [147]:
plot_ts(a, random_walk)

Loading BokehJS ...

### Задание 6. "Прогоните" все алгоритмы на всех рядах и получите сводную таблицу результатов по всем метрикам, постройте также графики прогнозов. 

In [35]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import r2_score

from metrics import mase

In [36]:
from estimators import RollingEstimator, ExponentialSmoothingEstimator
from pmdarima import auto_arima
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

## long

In [36]:
plot_ts(long)

Loading BokehJS ...

In [56]:
# def get_statistics(ts, granularity, num_lags, use_arima = True, period = 365, d = 1, D=1, seasonal=True):

In [49]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [50]:
granularity = 'P0Y0M1D'
ts = long

In [71]:
preds_in_sample = {}
preds_out_of_sample = {}

models = {'RollingEstimator': RollingEstimator, 'LinearRegression' :LinearRegression, 
          'ExponentialSmoothingEstimator': ExponentialSmoothingEstimator,
          'RandomForestRegressor': RandomForestRegressor, 'GradientBoostingRegressor': GradientBoostingRegressor}

results_in_sample = pd.DataFrame(columns = ['algorithm', 'mse', 'mae', 'mape', 'r2_score'])
results_out_of_sample = pd.DataFrame(columns = ['algorithm', 'mse', 'mae', 'mape', 'r2_score'])

In [72]:
for name, model in models.items():
    print(name)
    ts_train, ts_test, in_sample, out_of_sample = make_pipeline(ts, granularity, model, num_lags = 30)
    preds_in_sample[name] = in_sample
    preds_out_of_sample[name] = out_of_sample
    
    
    stats_in_sample = {'algorithm': name, 'mse': mse(ts_test, in_sample),
                       'mae': mae(ts_test, in_sample),
                       'mape': mape(ts_test, in_sample), 'r2_score': r2_score(ts_test, in_sample)}
    
    stats_out_of_sample = {'algorithm': name, 'mse': mse(ts_test, out_of_sample),
                           'mae': mae(ts_test, out_of_sample),
                           'mape': mape(ts_test, out_of_sample), 'r2_score': r2_score(ts_test, out_of_sample)}
    

    
    results_in_sample = pd.concat([results_in_sample, pd.DataFrame([stats_in_sample])])
    results_out_of_sample = pd.concat([results_out_of_sample, pd.DataFrame([stats_out_of_sample])])
    

RollingEstimator
LinearRegression
ExponentialSmoothingEstimator
RandomForestRegressor
GradientBoostingRegressor


In [65]:
#с arima здесь зависает ноутбук

In [66]:
# #arima
# ts_train, ts_test, in_sample, out_of_sample = make_pipeline_arima(long, granularity, period = 365,
#                                                             d = 0, D = 1, seasonal = True)

# stats_in_sample = {'algorithm': 'arima', 'mse': mse(ts_test, in_sample), 'mae': mae(ts_test, in_sample),
#                        'mape': mape(ts_test, in_sample), 'r2_score': r2_score(ts_test, in_sample)}
    
# stats_out_of_sample = {'algorithm': 'arima', 'mse': mse(ts_test, out_of_sample), 'mae': mae(ts_test, out_of_sample),
#                            'mape': mape(ts_test, out_of_sample), 'r2_score': r2_score(ts_test, out_of_sample)}

# preds_in_sample['arima'] = in_sample
# preds_out_of_sample['arima'] = out_of_sample

# results_in_sample = pd.concat([results_in_sample, pd.DataFrame([stats_in_sample])])
# results_out_of_sample = pd.concat([results_out_of_sample, pd.DataFrame([stats_out_of_sample])])

In [73]:
#fc_network
ts_train, ts_test, in_sample, out_of_sample = make_pipeline_fullyconnected(ts, 
                                                                           granularity = granularity,
                                                                           num_lags_in = 30,
                                                                           num_lags_out = 12,
                                                                           units = [64,32], epochs = 100)

stats_in_sample = {'algorithm': 'fc_network', 'mse': mse(ts_test, in_sample),
                   'mae': mae(ts_test, in_sample),
                    'mape': mape(ts_test, in_sample), 'r2_score': r2_score(ts_test, in_sample)}
    
stats_out_of_sample = {'algorithm': 'fc_network', 'mse': mse(ts_test, out_of_sample),
                       'mae': mae(ts_test, out_of_sample),
                       'mape': mape(ts_test, out_of_sample), 'r2_score': r2_score(ts_test, out_of_sample)}

preds_in_sample['fc_network'] = in_sample
preds_out_of_sample['fc_network'] = out_of_sample

results_in_sample = pd.concat([results_in_sample, pd.DataFrame([stats_in_sample])])
results_out_of_sample = pd.concat([results_out_of_sample, pd.DataFrame([stats_out_of_sample])])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                1984      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 12)                396       
                                                                 
Total params: 4,460
Trainable params: 4,460
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
79/79 [==============================] - 1s 2ms/step - loss: 28.7070
Epoch 2/100
79/79 [==============================] - 0s 2ms/step - loss: 9.5963
Epoch 3/100
79/79 [==============================] - 0s 2ms/step - loss: 9.3975
Epoch 4/100
79/79 [==============================] - 0s 2ms/step - los

79/79 [==============================] - 0s 2ms/step - loss: 8.1386
Epoch 93/100
79/79 [==============================] - 0s 2ms/step - loss: 8.2505
Epoch 94/100
79/79 [==============================] - 0s 2ms/step - loss: 8.1075
Epoch 95/100
79/79 [==============================] - 0s 2ms/step - loss: 8.1227
Epoch 96/100
79/79 [==============================] - 0s 2ms/step - loss: 8.1098
Epoch 97/100
79/79 [==============================] - 0s 2ms/step - loss: 8.1571
Epoch 98/100
79/79 [==============================] - 0s 2ms/step - loss: 8.1196
Epoch 99/100
79/79 [==============================] - 0s 2ms/step - loss: 8.1035
Epoch 100/100
79/79 [==============================] - 0s 2ms/step - loss: 8.1478


In [74]:
results_in_sample

,algorithm,mse,mae,mape,r2_score
0,RollingEstimator,15.842548,3.277477,0.291332,0.007078
0,LinearRegression,5.076071,1.788846,0.194816,0.681860
0,ExponentialSmoothingEstimator,136.887491,10.998425,0.935182,-7.579338
0,RandomForestRegressor,5.127266,1.791564,0.194061,0.678652
0,GradientBoostingRegressor,4.948948,1.756606,0.189142,0.689828
0,fc_network,7.976481,2.189979,0.249479,0.500079


In [88]:
results_out_of_sample

,algorithm,mse,mae,mape,r2_score
0,RollingEstimator,132.971115,10.861427,0.939628,-7.333882
0,LinearRegression,7.241509,2.102646,0.230017,0.546143
0,ExponentialSmoothingEstimator,137.145591,11.009149,0.936145,-7.595514
0,RandomForestRegressor,18.962551,3.533926,0.487021,-0.188466
0,GradientBoostingRegressor,12.044565,2.722041,0.364191,0.245114
0,fc_network,15.735812,3.256556,0.410865,0.013768


In [76]:
plot_ts(ts_train, ts_test, preds_out_of_sample['LinearRegression'])

Loading BokehJS ...

In [77]:
plot_ts(ts_train, ts_test, preds_out_of_sample['GradientBoostingRegressor'])

Loading BokehJS ...

In [89]:
plot_ts(ts_train, ts_test, preds_out_of_sample['RandomForestRegressor'])

Loading BokehJS ...

In [92]:
plot_ts(ts_train, ts_test, preds_out_of_sample['fc_network'])

Loading BokehJS ...

Здесь лучше всего линейня регрессия. Нейронная сеть выдает странные результаты. Возможно, она недообучилась. in_sample работает нормально

In [93]:
plot_ts(ts_train, ts_test, preds_in_sample['fc_network'])

Loading BokehJS ...

## short_season

In [94]:
short_season = ds['hour_3019.csv'][300:]

In [95]:
plot_ts(short_season)

Loading BokehJS ...

In [96]:
short_season

2019-05-30T10:00:00
2019-06-13 09:00:00     71.0
2019-06-13 10:00:00    108.0
2019-06-13 11:00:00    117.0
2019-06-13 12:00:00    119.0
2019-06-13 13:00:00    116.0
                       ...  
2019-07-02 14:00:00    132.0
2019-07-02 15:00:00    128.0
2019-07-02 16:00:00     98.0
2019-07-02 17:00:00    105.0
2019-07-02 18:00:00    107.0
Name: 96.0, Length: 381, dtype: float64

In [97]:
# P(n)Y(n)M(n)DT(n)H(n)M(n)S

In [98]:
granularity = 'P0Y0M0DT1H'
ts = short_season

In [99]:
preds_in_sample = {}
preds_out_of_sample = {}

models = {'RollingEstimator': RollingEstimator, 'LinearRegression' :LinearRegression, 
          'ExponentialSmoothingEstimator': ExponentialSmoothingEstimator,
          'RandomForestRegressor': RandomForestRegressor, 'GradientBoostingRegressor': GradientBoostingRegressor}

results_in_sample = pd.DataFrame(columns = ['algorithm', 'mse', 'mae', 'mape', 'r2_score'])
results_out_of_sample = pd.DataFrame(columns = ['algorithm', 'mse', 'mae', 'mape', 'r2_score'])

In [100]:
for name, model in models.items():
    print(name)
    ts_train, ts_test, in_sample, out_of_sample = make_pipeline(ts, granularity, model, num_lags = 30)
    preds_in_sample[name] = in_sample
    preds_out_of_sample[name] = out_of_sample
    
    
    stats_in_sample = {'algorithm': name, 'mse': mse(ts_test, in_sample),
                       'mae': mae(ts_test, in_sample),
                       'mape': mape(ts_test, in_sample), 'r2_score': r2_score(ts_test, in_sample)}
    
    stats_out_of_sample = {'algorithm': name, 'mse': mse(ts_test, out_of_sample),
                           'mae': mae(ts_test, out_of_sample),
                           'mape': mape(ts_test, out_of_sample), 'r2_score': r2_score(ts_test, out_of_sample)}
    

    
    results_in_sample = pd.concat([results_in_sample, pd.DataFrame([stats_in_sample])])
    results_out_of_sample = pd.concat([results_out_of_sample, pd.DataFrame([stats_out_of_sample])])
    

RollingEstimator
LinearRegression
ExponentialSmoothingEstimator
RandomForestRegressor
GradientBoostingRegressor


In [107]:
#arima
ts_train, ts_test, in_sample, out_of_sample = make_pipeline_arima(ts, granularity, period = 24,
                                                            d = 0, D = 1, seasonal = True)

In [114]:
stats_in_sample = {'algorithm': 'arima', 'mse': mse(ts_test, in_sample), 'mae': mae(ts_test, in_sample),
                       'mape': mape(ts_test, in_sample), 'r2_score': r2_score(ts_test, in_sample)}
    
stats_out_of_sample = {'algorithm': 'arima', 'mse': mse(ts_test, out_of_sample[ts_test.index]),
                       'mae': mae(ts_test, out_of_sample[ts_test.index]),
                           'mape': mape(ts_test, out_of_sample[ts_test.index]),
                       'r2_score': r2_score(ts_test, out_of_sample[ts_test.index])}

preds_in_sample['arima'] = in_sample
preds_out_of_sample['arima'] = out_of_sample

results_in_sample = pd.concat([results_in_sample, pd.DataFrame([stats_in_sample])])
results_out_of_sample = pd.concat([results_out_of_sample, pd.DataFrame([stats_out_of_sample])])

In [117]:
#fc_network
ts_train, ts_test, in_sample, out_of_sample = make_pipeline_fullyconnected(ts, 
                                                                           granularity = granularity,
                                                                           num_lags_in = 30,
                                                                           num_lags_out = 12,
                                                                           units = [64,32], epochs = 2000)

stats_in_sample = {'algorithm': 'fc_network', 'mse': mse(ts_test, in_sample),
                   'mae': mae(ts_test, in_sample),
                    'mape': mape(ts_test, in_sample), 'r2_score': r2_score(ts_test, in_sample)}
    
stats_out_of_sample = {'algorithm': 'fc_network', 'mse': mse(ts_test, out_of_sample),
                       'mae': mae(ts_test, out_of_sample),
                       'mape': mape(ts_test, out_of_sample), 'r2_score': r2_score(ts_test, out_of_sample)}

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 64)                1984      
                                                                 
 dense_25 (Dense)            (None, 32)                2080      
                                                                 
 dense_26 (Dense)            (None, 12)                396       
                                                                 
Total params: 4,460
Trainable params: 4,460
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2000
8/8 [==============================] - 1s 3ms/step - loss: 11333.8311
Epoch 2/2000
8/8 [==============================] - 0s 3ms/step - loss: 7493.9570
Epoch 3/2000
8/8 [==============================] - 0s 3ms/step - loss: 5540.1885
Epoch 4/2000
8/8 [==============================] - 0s 3ms/step 

8/8 [==============================] - 0s 3ms/step - loss: 352.9564
Epoch 91/2000
8/8 [==============================] - 0s 3ms/step - loss: 351.6616
Epoch 92/2000
8/8 [==============================] - 0s 3ms/step - loss: 343.0129
Epoch 93/2000
8/8 [==============================] - 0s 3ms/step - loss: 343.3507
Epoch 94/2000
8/8 [==============================] - 0s 3ms/step - loss: 343.6022
Epoch 95/2000
8/8 [==============================] - 0s 2ms/step - loss: 352.5192
Epoch 96/2000
8/8 [==============================] - 0s 3ms/step - loss: 355.5031
Epoch 97/2000
8/8 [==============================] - 0s 3ms/step - loss: 337.8697
Epoch 98/2000
8/8 [==============================] - 0s 3ms/step - loss: 344.7105
Epoch 99/2000
8/8 [==============================] - 0s 3ms/step - loss: 345.1450
Epoch 100/2000
8/8 [==============================] - 0s 2ms/step - loss: 334.2286
Epoch 101/2000
8/8 [==============================] - 0s 3ms/step - loss: 336.6666
Epoch 102/2000
8/8 [========

Epoch 189/2000
8/8 [==============================] - 0s 3ms/step - loss: 304.3485
Epoch 190/2000
8/8 [==============================] - 0s 3ms/step - loss: 332.5420
Epoch 191/2000
8/8 [==============================] - 0s 2ms/step - loss: 290.4765
Epoch 192/2000
8/8 [==============================] - 0s 3ms/step - loss: 294.1468
Epoch 193/2000
8/8 [==============================] - 0s 3ms/step - loss: 277.0458
Epoch 194/2000
8/8 [==============================] - 0s 3ms/step - loss: 281.4642
Epoch 195/2000
8/8 [==============================] - 0s 3ms/step - loss: 275.1596
Epoch 196/2000
8/8 [==============================] - 0s 3ms/step - loss: 270.7383
Epoch 197/2000
8/8 [==============================] - 0s 3ms/step - loss: 270.2653
Epoch 198/2000
8/8 [==============================] - 0s 2ms/step - loss: 272.1422
Epoch 199/2000
8/8 [==============================] - 0s 2ms/step - loss: 267.2056
Epoch 200/2000
8/8 [==============================] - 0s 2ms/step - loss: 278.4364
Epoc

8/8 [==============================] - 0s 3ms/step - loss: 261.6481
Epoch 288/2000
8/8 [==============================] - 0s 3ms/step - loss: 247.1119
Epoch 289/2000
8/8 [==============================] - 0s 3ms/step - loss: 243.5473
Epoch 290/2000
8/8 [==============================] - 0s 3ms/step - loss: 247.0827
Epoch 291/2000
8/8 [==============================] - 0s 3ms/step - loss: 248.6926
Epoch 292/2000
8/8 [==============================] - 0s 3ms/step - loss: 239.1914
Epoch 293/2000
8/8 [==============================] - 0s 3ms/step - loss: 248.0694
Epoch 294/2000
8/8 [==============================] - 0s 3ms/step - loss: 239.3287
Epoch 295/2000
8/8 [==============================] - 0s 3ms/step - loss: 234.9851
Epoch 296/2000
8/8 [==============================] - 0s 3ms/step - loss: 239.6305
Epoch 297/2000
8/8 [==============================] - 0s 3ms/step - loss: 236.8264
Epoch 298/2000
8/8 [==============================] - 0s 3ms/step - loss: 236.3476
Epoch 299/2000
8/8 

8/8 [==============================] - 0s 3ms/step - loss: 224.6037
Epoch 386/2000
8/8 [==============================] - 0s 2ms/step - loss: 226.2262
Epoch 387/2000
8/8 [==============================] - 0s 3ms/step - loss: 220.9238
Epoch 388/2000
8/8 [==============================] - 0s 3ms/step - loss: 223.8137
Epoch 389/2000
8/8 [==============================] - 0s 3ms/step - loss: 222.1933
Epoch 390/2000
8/8 [==============================] - 0s 3ms/step - loss: 220.6172
Epoch 391/2000
8/8 [==============================] - 0s 3ms/step - loss: 218.7852
Epoch 392/2000
8/8 [==============================] - 0s 3ms/step - loss: 227.3370
Epoch 393/2000
8/8 [==============================] - 0s 2ms/step - loss: 230.2429
Epoch 394/2000
8/8 [==============================] - 0s 2ms/step - loss: 224.5592
Epoch 395/2000
8/8 [==============================] - 0s 3ms/step - loss: 251.5612
Epoch 396/2000
8/8 [==============================] - 0s 2ms/step - loss: 239.0351
Epoch 397/2000
8/8 

...

8/8 [==============================] - 0s 3ms/step - loss: 101.0920
Epoch 1557/2000
8/8 [==============================] - 0s 3ms/step - loss: 101.2140
Epoch 1558/2000
8/8 [==============================] - 0s 3ms/step - loss: 99.5724
Epoch 1559/2000
8/8 [==============================] - 0s 3ms/step - loss: 110.6501
Epoch 1560/2000
8/8 [==============================] - 0s 3ms/step - loss: 120.7226
Epoch 1561/2000
8/8 [==============================] - 0s 3ms/step - loss: 107.3739
Epoch 1562/2000
8/8 [==============================] - 0s 2ms/step - loss: 109.6471
Epoch 1563/2000
8/8 [==============================] - 0s 3ms/step - loss: 111.9641
Epoch 1564/2000
8/8 [==============================] - 0s 3ms/step - loss: 106.8715
Epoch 1565/2000
8/8 [==============================] - 0s 3ms/step - loss: 104.3352
Epoch 1566/2000
8/8 [==============================] - 0s 3ms/step - loss: 120.7557
Epoch 1567/2000
8/8 [==============================] - 0s 3ms/step - loss: 106.7134
Epoch 156

Epoch 1654/2000
8/8 [==============================] - 0s 3ms/step - loss: 96.1870
Epoch 1655/2000
8/8 [==============================] - 0s 3ms/step - loss: 118.5365
Epoch 1656/2000
8/8 [==============================] - 0s 3ms/step - loss: 123.2385
Epoch 1657/2000
8/8 [==============================] - 0s 3ms/step - loss: 112.8714
Epoch 1658/2000
8/8 [==============================] - 0s 3ms/step - loss: 106.3966
Epoch 1659/2000
8/8 [==============================] - 0s 3ms/step - loss: 98.8394
Epoch 1660/2000
8/8 [==============================] - 0s 3ms/step - loss: 95.0363
Epoch 1661/2000
8/8 [==============================] - 0s 3ms/step - loss: 99.5022
Epoch 1662/2000
8/8 [==============================] - 0s 3ms/step - loss: 95.5697
Epoch 1663/2000
8/8 [==============================] - 0s 2ms/step - loss: 102.7713
Epoch 1664/2000
8/8 [==============================] - 0s 3ms/step - loss: 99.8663
Epoch 1665/2000
8/8 [==============================] - 0s 3ms/step - loss: 98.5703

8/8 [==============================] - 0s 3ms/step - loss: 96.2437
Epoch 1753/2000
8/8 [==============================] - 0s 3ms/step - loss: 90.7355
Epoch 1754/2000
8/8 [==============================] - 0s 3ms/step - loss: 89.9821
Epoch 1755/2000
8/8 [==============================] - 0s 2ms/step - loss: 99.8952
Epoch 1756/2000
8/8 [==============================] - 0s 3ms/step - loss: 98.8154
Epoch 1757/2000
8/8 [==============================] - 0s 3ms/step - loss: 96.6394
Epoch 1758/2000
8/8 [==============================] - 0s 3ms/step - loss: 114.0592
Epoch 1759/2000
8/8 [==============================] - 0s 3ms/step - loss: 107.6367
Epoch 1760/2000
8/8 [==============================] - 0s 3ms/step - loss: 99.2540
Epoch 1761/2000
8/8 [==============================] - 0s 3ms/step - loss: 108.8832
Epoch 1762/2000
8/8 [==============================] - 0s 3ms/step - loss: 102.8476
Epoch 1763/2000
8/8 [==============================] - 0s 3ms/step - loss: 99.5152
Epoch 1764/2000


8/8 [==============================] - 0s 3ms/step - loss: 90.9394
Epoch 1851/2000
8/8 [==============================] - 0s 3ms/step - loss: 89.3534
Epoch 1852/2000
8/8 [==============================] - 0s 3ms/step - loss: 86.3826
Epoch 1853/2000
8/8 [==============================] - 0s 3ms/step - loss: 84.3376
Epoch 1854/2000
8/8 [==============================] - 0s 3ms/step - loss: 82.2214
Epoch 1855/2000
8/8 [==============================] - 0s 3ms/step - loss: 81.5134
Epoch 1856/2000
8/8 [==============================] - 0s 3ms/step - loss: 81.7901
Epoch 1857/2000
8/8 [==============================] - 0s 3ms/step - loss: 80.4825
Epoch 1858/2000
8/8 [==============================] - 0s 3ms/step - loss: 85.3075
Epoch 1859/2000
8/8 [==============================] - 0s 3ms/step - loss: 82.3435
Epoch 1860/2000
8/8 [==============================] - 0s 3ms/step - loss: 80.0800
Epoch 1861/2000
8/8 [==============================] - 0s 3ms/step - loss: 78.9296
Epoch 1862/2000
8/8 

8/8 [==============================] - 0s 3ms/step - loss: 95.4221
Epoch 1949/2000
8/8 [==============================] - 0s 3ms/step - loss: 90.3888
Epoch 1950/2000
8/8 [==============================] - 0s 3ms/step - loss: 85.1794
Epoch 1951/2000
8/8 [==============================] - 0s 3ms/step - loss: 88.9618
Epoch 1952/2000
8/8 [==============================] - 0s 3ms/step - loss: 96.7574
Epoch 1953/2000
8/8 [==============================] - 0s 3ms/step - loss: 89.2959
Epoch 1954/2000
8/8 [==============================] - 0s 2ms/step - loss: 85.5206
Epoch 1955/2000
8/8 [==============================] - 0s 3ms/step - loss: 86.9102
Epoch 1956/2000
8/8 [==============================] - 0s 2ms/step - loss: 86.5666
Epoch 1957/2000
8/8 [==============================] - 0s 2ms/step - loss: 82.9140
Epoch 1958/2000
8/8 [==============================] - 0s 3ms/step - loss: 82.7381
Epoch 1959/2000
8/8 [==============================] - 0s 3ms/step - loss: 93.8145
Epoch 1960/2000
8/8 

In [128]:
preds_in_sample['fc_network'] = in_sample
preds_out_of_sample['fc_network'] = out_of_sample

In [121]:
results_in_sample = pd.concat([results_in_sample, pd.DataFrame([stats_in_sample])])
results_out_of_sample = pd.concat([results_out_of_sample, pd.DataFrame([stats_out_of_sample])])

In [122]:
results_in_sample

,algorithm,mse,mae,mape,r2_score
0,RollingEstimator,3813.615602,52.904747,4.052572,-0.201534
0,LinearRegression,633.755099,17.840848,1.177619,0.800326
0,ExponentialSmoothingEstimator,8628.868788,73.824946,0.952426,-1.718648
0,RandomForestRegressor,432.968196,12.534522,0.609165,0.863587
0,GradientBoostingRegressor,430.654443,12.664006,0.799479,0.864316
0,arima,558.874897,16.234318,0.857523,0.823919
0,fc_network,694.357355,17.936520,0.821885,0.781233


In [123]:
results_out_of_sample

,algorithm,mse,mae,mape,r2_score
0,RollingEstimator,6193.402785,61.330963,1.992757,-0.951320
0,LinearRegression,2636.249180,41.983866,2.606493,0.169412
0,ExponentialSmoothingEstimator,8643.567656,73.895594,0.954763,-1.723279
0,RandomForestRegressor,3871.039580,52.644261,5.284013,-0.219626
0,GradientBoostingRegressor,3551.874089,50.225364,4.948285,-0.119069
0,arima,4651.094572,57.999641,5.889995,-0.465394
0,fc_network,7156.602054,70.858146,6.758120,-1.254789


In [124]:
plot_ts(ts_train, ts_test, preds_out_of_sample['LinearRegression'])

Loading BokehJS ...

In [125]:
plot_ts(ts_train, ts_test, preds_out_of_sample['GradientBoostingRegressor'])

Loading BokehJS ...

In [126]:
plot_ts(ts_train, ts_test, preds_out_of_sample['RandomForestRegressor'])

Loading BokehJS ...

In [129]:
plot_ts(ts_train, ts_test, preds_out_of_sample['fc_network'])

Loading BokehJS ...

In [130]:
plot_ts(ts_train, ts_test, preds_out_of_sample['arima'])

Loading BokehJS ...

Нейронная сеть на этот раз дает результаты получе.\
Самое лучшее качество опять показала LinearRegression, а данных случаях она меньше всего склонна к переобучению.

In [131]:
#дальше не успел

In [12]:
short_season_trend = ds['international-airline-passengers.csv']

/home/nikita/Документы/time_series_course/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [13]:
plot_ts(short_season_trend)

Loading BokehJS ...

In [14]:
random_walk = ds['dow_jones_0.csv']

/home/nikita/Документы/time_series_course/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [15]:
plot_ts(random_walk)

Loading BokehJS ...

In [16]:
medium_noize = ds['hour_3426.csv'][300:]

/home/nikita/Документы/time_series_course/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [17]:
plot_ts(medium_noize)

Loading BokehJS ...

In [18]:
full_noize = ds['day_1574.csv']

/home/nikita/Документы/time_series_course/ts_march_2023/load_dataset.py:34: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  ts = pd.read_csv(


In [19]:
plot_ts(full_noize)

Loading BokehJS ...